## Demo of cryostat builder
This notebook contains a quick demo of the Cryostat class that makes the task of adding a cryostat model much less daunting. The system uses McStas Union components, and for the moment only supports McStas 2.X.

We start the demo by creating an instrument object with just a source.

In [ ]:
import mcstasscript as ms
import mcstasscript.jb_interface as ms_widget

instrument = ms.McStas_instr("cryostat_demo")

In [ ]:
source = instrument.add_component("source", "Source_simple")
source.xwidth = 0.01
source.yheight = 0.01
source.focus_xw = 0.01
source.focus_yh = 0.01
source.dist = 2
source.E0 = 5
source.dE = 0.1

## Creating a cryostat
One use the Cryostat class to make a cryostat object. This object can be placed in the instrument file much like a component with *set_AT* and *set_ROTATED*. Here we place the cryostat origin 2 m after the source.

In [ ]:
orange_cryostat = ms.Cryostat("orange", instrument)
orange_cryostat.set_AT([0,0,2], source)

The cryostat model is constructed in layers, each of which are added with *add_layer*. The geometry is shown in the left side of the figure below. To define the walls the user specifies two of *inner_radius*, *outer_radius* and *thickness*, where the last one will be calculated. The distance to the top is controlled with *origin_to_top*, ands its thickness with *top_thickness*. A small negative *top_thickness* will remove the top. The bottom is controlled with *origin_to_bottom* and *bottom_thickness*.

One can set *p_interact* which controls the fraction of the rays that scatters in this layer. This value should be kept below 0.5, but since cryostats are supposed to be transparent even setting 0.2 greatly increases the statistics gain from the simulation.

![title](cryostat_image.png)

Here the first of 4 layers are added, the layers have to be added from smallest to largest.

In [ ]:
orange_cryostat.add_layer(inner_radius=70E-3/2, outer_radius=75E-3/2,
                          origin_to_top=200E-3, top_thickness=-1E-3,
                          origin_to_bottom=83E-3, bottom_thickness=5E-3,
                          p_interact=0.2)

After adding a layer, it can be accessed with the attribute *last_layer*, which allows adding a window. The geometry of a window is shown to the right in the figure above. To control the height of the window, the user provides two of *height*, *origin_to_top* and *origin_to_bottom*. If the window only changes the inner radius, only specify the *inner_radius* parameter, and vice versa with the *outer_radius*. If the window narrows the cryostat from both sides, the user should specify two of *inner_radius*, *outer_radius* and *thickness*. It is not allowed to specify *inner_radius* or *outer_radius* that equals the corresponding value for the layer.

Here a simple window is added to the last layer.

In [ ]:
orange_cryostat.last_layer.add_window(outer_radius=73E-3/2, origin_to_top=44.42E-3, origin_to_bottom=88.2E-3)

It is allowed to add multiple windows to a layer, this could for example be needed when describing a window where the outside and inside have different heights.

In [ ]:
orange_cryostat.last_layer.add_window(inner_radius=71E-3/2, height=40E-3)

If "Al" is selected for material, a simple Al model is created unless an Al model already exists in the instrument. When using the material keyword to add other materials, it is up to the user to create the material definition using Union components. The inside is always Vacuum. Here the second layer is added, specifying "Al" as the material.

In [ ]:
orange_cryostat.add_layer(inner_radius=80E-3/2, outer_radius=81E-3/2,
                          origin_to_bottom=90E-3, bottom_thickness=2E-3, 
                          origin_to_top=240E-3, top_thickness=-1E-3,
                          material="Al", p_interact=0.2)

The remaining 2 of the 4 layers are added here, both with windows.

In [ ]:
orange_cryostat.add_layer(inner_radius=95E-3/2, outer_radius=99.5E-3/2,
                          origin_to_bottom=93E-3, bottom_thickness=6E-3,
                          origin_to_top=225E-3, top_thickness=9E-3, p_interact=0.2)
orange_cryostat.last_layer.add_window(outer_radius=97E-3/2, origin_to_top=52E-3, origin_to_bottom=100E-3)

orange_cryostat.add_layer(inner_radius=120E-3/2, outer_radius=127E-3/2,
                          origin_to_bottom=109E-3, bottom_thickness=11E-3,
                          origin_to_top=205E-3, top_thickness=22E-3, p_interact=0.2)
orange_cryostat.last_layer.add_window(outer_radius=125E-3/2, inner_radius=122E-3/2,
                                      origin_to_top=55.7E-3, origin_to_bottom=93.54E-3)

After the cryostat description is done one can optionally add Union loggers that show scattering intensity in space using the *add_spatial_loggers* method. Shows the cryostat from the 3 directions along axis, and a zoom in on a cut in zy that clearly shows all windows added. In addition its possible to record the scattering as a function of time, this is done with the *add_time_histogram* and *add_animation* methods, the first of which adds a simple histogram and the latter of which records spatial scattering in a number of time frames.

In [ ]:
orange_cryostat.add_spatial_loggers()
orange_cryostat.add_time_histogram(t_min=0.00195, t_max=0.0024)
orange_cryostat.add_animation(t_min=0.00195, t_max=0.0024, n_frames=5)

At the end it is necessary to run the *build* method, this assigns the appropriate priorities to each Union component used, and adds a Union_master component to the end. Per default the build method adds a *Union_master* component, if one wants to for example add a sample afterwards, this is turned of by setting include_master to False.

In [ ]:
orange_cryostat.build(include_master=True)

For further information on these methods, the built in help can be used shown below

In [ ]:
help(orange_cryostat.add_animation)

The build method adds a large number of components to describe the cryostat. Lets see what this looks like.

In [ ]:
instrument.show_components()

## Displaying the geometry

In [ ]:
instrument.show_instrument()

### Running the simulation
This is sufficient to run the simulation and see the resulting plots.

In [ ]:
instrument.settings(ncount=1E7)
empty_data = instrument.backengine()

### Plotting with interface
Recommend using log plot and orders of magnitude = 5 to see details. It may be necessary to refresh this cell after the simulation has been performed. Use "zy_close" to get the clearest view of the the added layers and windows.

In [ ]:
%matplotlib widget
ms_widget.show(empty_data)

The "logger_space_zy_time" monitor can be animated with the built in *make_animation* function.

In [ ]:
ani_data = empty_data[5:10]
for frame in ani_data:
    frame.set_plot_options(log=True, colormap="hot", orders_of_mag=6)
    
ms.make_animation(ani_data, filename="cryostat_animation", fps=2, figsize=(10,6))

## Adding a sample
It is most natural to add a Union sample to the cryostat, and this could be done by not adding a Union_master component with the cryostat *build* method, and then adding the Union_master after the sample. It can however also be done by inserting the necessary components at the right place in the instrument using the *before* keyword.

Here a powder sample is added and the simulation is executed again. Notice the sample location is set to the same as the cryostat name, this refers to the arm used as a reference for the cryostat sample position.

In [ ]:
nascalf_inc = instrument.add_component("nascalf_inc", "Incoherent_process", before="Al_inc")
nascalf_inc.sigma = 3.4176
nascalf_inc.unit_cell_volume = 1079.1

nascalf_pow = instrument.add_component("nascalf_pow", "Powder_process", before="Al_inc")
nascalf_pow.reflections = '"Na2Ca3Al2F14.laz"'

nascalf = instrument.add_component("nascalf", "Union_make_material", before="Al_inc")
nascalf.process_string = '"nascalf_inc,nascalf_pow"'
nascalf.my_absorption = 100*2.9464/1079.1

sample = instrument.add_component("sample", "Union_cylinder", before="orange_master")
sample.radius = 6E-3
sample.yheight = 30E-3
sample.priority = 1000
sample.material_string = '"nascalf"'
sample.p_interact = 0.3
sample.set_AT([0,0,0], RELATIVE="orange")

In [ ]:
instrument.settings(ncount=2E7, mpi=2)
data_sample = instrument.backengine()

In [ ]:
ms_widget.show(data_sample)

## Future expansions
Some additional features are expected to be added to this system at a later point.

### Entry windows
Create a layer method to make entry windows, square and circular.

### Mounting plate
A cryostat usually has a mounting plate with a larger radius than the widest layer, could easily add such a feature.

### External sample
Create a cryostat method that takes an external sample component that is not in Union but incorperates this into the cryostat using the best practice method.